### 1. 라이브러리 & 데이터 리딩

In [ ]:
# 수치화를 위한 라이브러리 
import pandas as pd 
import numpy as np 

# 시각화를 위한 라이브러리 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 한글 깨짐 방지
from matplotlib import rc
rc('font', family ='AppleGothic')

In [ ]:
jeju_reg_17_df = pd.read_csv('data-2/jeju_card_region_2017.csv')
jeju_reg_18_df = pd.read_csv('data-2/jeju_card_region_2018.csv')
jeju_pop_df = pd.read_csv('data-2/jeju_population.csv')

### 2. 결측치 및 칼럼처리 

#### 결측치 및 unique 값 확인

In [ ]:
print(jeju_reg_17_df.shape)
print(jeju_reg_18_df.shape)
print(jeju_pop_df.shape)

In [ ]:
jeju_reg_17_df.head()

In [ ]:
jeju_reg_18_df.head()

In [ ]:
jeju_pop_df.head()
# 연월방문 인구 데이터 

In [ ]:
print(jeju_pop_df.loc[527025])
# 마지막  2018년 12월 31일 자료 

In [ ]:
jeju_reg_17_df.info()

In [ ]:
jeju_reg_17_df.isnull().sum()

결측치 없음 

In [ ]:
jeju_reg_18_df.info()

In [ ]:
jeju_reg_18_df.isnull().sum()

결측치 없음 

In [ ]:
jeju_pop_df.info()

In [ ]:
jeju_pop_df.isnull().sum()

결측치 없음 

In [ ]:
#### 2017년, 2018년 데이터 비교 및 전처리 
#이를 위해, 각 DataFrame에 있는 object 타입 컬럼이 어떤 값으로 이루어져 있는지 비교해 볼 건데요. 아래와 같이  
#print_unique_values()라는 함수를 만들어서 쓸 겁니다. 이 함수를 통해 각 컬럼의 unique 한 값의 개수와 종류를 출력해서 볼 수 있어요.

def unique_value(df):
    non_numeric_cols = df.columns[df.dtypes == 'object'] # Object 값만 추출 
    for col in non_numeric_cols:
        print(f'{col} unique 값: {df[col].nunique()}')
        # print(f'{col} 컬럼의 unique 값 개수: {df[col].nunique()}')
        print(sorted(df[col].unique()), '\n')
    

In [ ]:
unique_value(jeju_reg_17_df)

In [ ]:
unique_value(jeju_reg_18_df)

In [ ]:
unique_value(jeju_pop_df)

#### 업종명: 2017년과 2018년도 unique 값 비교 & drop

In [ ]:
# 2017에만 있는 값
for item in jeju_reg_17_df['업종명'].unique():
    if item not in jeju_reg_18_df['업종명'].unique():
        print(f'only 2017: {item}')
        
# 2018에만 있는 값
for item in jeju_reg_18_df['업종명'].unique():
    if item not in jeju_reg_17_df['업종명'].unique():
        print(f'only 2018: {item}')        

In [ ]:
# 개수 확인하고 제외하기 
print(jeju_reg_17_df[jeju_reg_17_df['업종명'] == '기타 갬블링 및 배팅업'].shape) # 왜 0이 나올까~~ 
print(jeju_reg_18_df[jeju_reg_18_df['업종명'] == '택시 운송업'].shape)


In [ ]:
# 갬블링, 택시 운송업 지우기 
jeju_reg_17_df = jeju_reg_17_df[jeju_reg_17_df['업종명'] != '기타 갬블링 및 베팅업']
jeju_reg_18_df = jeju_reg_18_df[jeju_reg_18_df['업종명'] != '택시 운송업']


#### 데이터 프레입 합치기

2017년도 df와 2018년도 df를 하나의 df로 합치기

In [ ]:
df = pd.concat([jeju_reg_18_df, jeju_reg_17_df])
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
df.shape

#### 연월 str 슬라이싱

In [ ]:
df['연월'] = df['연월'].str[:7]
df.head()

### 3. 데이터 분석

#### 3-1. 카드 이용이 많은 업종 확인하기

In [ ]:
#일단 이용 금액이 많은 업종은 무엇인지 알아볼 건데요. 
usebybiz = df.groupby('업종명').sum().reset_index()
usebybiz.sort_values(by='이용금액', ascending = False).head()

- 한식 음식점업이 이용금액이 가장 크다. 

In [ ]:
# 이용자수이 많은 업종은 무엇인지 알아볼 건데요. 
usebybiz.sort_values(by='이용자수', ascending = False).head()

- 이용자수가 가장 많은 업종은 체인화 편의점

In [ ]:
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
# 1인당 사용금애기 높은 업종 확인하기 
usebybiz['인당 이용금액'] = usebybiz['이용금액'] / usebybiz['이용자수']
usebybiz.sort_values(by='인당 이용금액', ascending = False).head()

버스 운송업은 데이터 프레임에서 제외

In [ ]:
df = df[df['업종명'] != '버스 운송업']
usebybiz = df.groupby('업종명').sum().reset_index()
usebybiz['인당 이용금액'] = usebybiz['이용금액'] / usebybiz['이용자수']

#상위 10개 업종 추출
usebybiz.sort_values(by='인당 이용금액', ascending = False).head(10)

In [ ]:
# 업종 별 이용금액 시각화 
plt.figure(figsize=(15,8))
sns.barplot(data = usebybiz, x = '업종명', y = '이용금액', palette = 'Set2')
plt.xticks(rotation = 90)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/100000000:,.0f}'))
plt.ylabel('이용금액(억)')
plt.show()

#### 3-2. 카드 이용이 많은 지역 확인하기 

In [ ]:
usebyreg = df.groupby('읍면동명').sum().reset_index()
usebyreg.head()

In [ ]:
usebyreg.sort_values(by='이용금액', ascending=False).head(10)

- 연동, 용담2동, 노형동, 송산동, 이도2동이 카드사용 상위 5위를 차지함 

In [ ]:
usebyreg.sort_values(by='이용자수', ascending=False).head(10)

- 이용금액과 다르게 이용자수는 노형동, 연동, 이도2동, 용담2동, 송산동이 상위 5위를 차지함 
- 순서는 다르지만 상위 5위 후보는 같음 

1인당 소비 금액이 많은 지역을 확인하자!

In [ ]:
usebyreg['인당 이용금액'] = usebyreg['이용금액'] / usebyreg['이용자수']
usebyreg.sort_values(by='인당 이용금액', ascending = False).head()

- 예래동, 영천동, 용담2동, 연동, 일도1동이 상위 5위를 차지함

In [ ]:
# 240302 카드 사용수가 많은 지역까지 작업함 
# 업데이트 되라 제발.. 
